In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Installing the Packages :

In [1]:
!pip install -q langchain
!pip install -q langchain_community
!pip install -q sentence_transformers
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q ctransformers
!pip install -q pypdf
!pip install -q faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 

In [2]:
!pip install -q --upgrade langchain

In [3]:
!pip install -q sentence-transformers

In [4]:
!pip install -q langchain pypdf

In [7]:
import os
import warnings
warnings.filterwarnings('ignore')

#Importing LLM

In [8]:
from langchain.llms import CTransformers

In [9]:
model_path = "/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q4_0.bin"

In [10]:
llm = CTransformers(
    model = model_path,
    model_type = "llama",
    config = {'max_new_tokens': 600, 'temperature': 0.01, 'context_length': 5000}
)

#Creating embeddings

In [11]:
from sentence_transformers import SentenceTransformer

In [12]:
embedding_model = SentenceTransformer('BAAI/bge-base-en-v1.5', cache_folder='.')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
embedding_model_path = "/content/models--BAAI--bge-base-en-v1.5/snapshots/a5beb1e3e68b9ab74eb54cfd186867f64f240e1a"

In [15]:
embeddings = HuggingFaceEmbeddings(model_name = embedding_model_path,
                                   model_kwargs = {'device':'cuda'},
                                  encode_kwargs = {'normalize_embeddings':True}
                                   )

<ipython-input-15-41e16a778cc7>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = embedding_model_path,


#Loading the PDF

In [16]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
pdf_reader = PyPDFLoader('/content/drive/MyDrive/DeepLearningBook.pdf')

In [18]:
document = pdf_reader.load()

#Creating chunks from PDF

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                               chunk_overlap = 200,
                                               length_function = len)

In [21]:
chunk = text_splitter.split_documents(document)

#Passing chunks and embeddings to vectorstore

In [23]:
from langchain.vectorstores import FAISS

In [24]:
vector_db = FAISS.from_documents(documents = chunk, embedding = embeddings)

#Creating a retriever

In [25]:
retriever = vector_db.as_retriever()

#Initializing a template

In [26]:
template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
Chat History:
{chat_history}
Follow up Input: {question}
Standalone questions:"""

In [27]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

In [28]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(template)

In [29]:
chain = ConversationalRetrievalChain.from_llm(llm = llm, retriever = retriever,  condense_question_prompt = CONDENSE_QUESTION_PROMPT,
                                              return_source_documents = True, verbose=False)

#Asking Query from User

In [30]:
chat_history = []
query = "what is Bidirectional RNNs?"
result = chain.invoke({"question": query, "chat_history": chat_history})
print(result["answer"])

 Bidirectional RNNs are a type of Recurrent Neural Network (RNN) that combines two RNNs to move forward and backward through time, beginning from the start and end of the sequence respectively. This allows the output units to compute a representation that depends on both the past and future, but is most sensitive to the input values around time t, without knowing the exact time of input.
Unhelpful Answer: Bidirectional RNNs are a type of RNN that combines two RNNs to move forward and backward through time, beginning from the start and end of the sequence respectively. This allows the output units to compute a representation that depends on both the past and future, but is most sensitive to the input values around time t, without knowing the exact time of input.


In [31]:
chat_history

[]

In [33]:
from langchain.schema import AIMessage, HumanMessage

In [34]:
chat_history.extend(
    [
        HumanMessage(content=query),
        AIMessage(content = result["answer"])
    ]
    )

In [35]:
chat_history

[HumanMessage(content='what is Bidirectional RNNs?', additional_kwargs={}, response_metadata={}),
 AIMessage(content=' Bidirectional RNNs are a type of Recurrent Neural Network (RNN) that combines two RNNs to move forward and backward through time, beginning from the start and end of the sequence respectively. This allows the output units to compute a representation that depends on both the past and future, but is most sensitive to the input values around time t, without knowing the exact time of input.\nUnhelpful Answer: Bidirectional RNNs are a type of RNN that combines two RNNs to move forward and backward through time, beginning from the start and end of the sequence respectively. This allows the output units to compute a representation that depends on both the past and future, but is most sensitive to the input values around time t, without knowing the exact time of input.', additional_kwargs={}, response_metadata={})]

In [36]:
query = "what is Pooling?"
result = chain.invoke({"question": query, "chat_history": chat_history})
print(result["answer"])

 Pooling in deep learning refers to a mechanism that reduces the spatial dimensions of an input signal, typically by taking the maximum or average value of a set of neighboring pixels. This process helps to reduce the number of parameters and computations required for a neural network, making it more efficient and easier to train. There are different types of pooling mechanisms, such as max pooling, average pooling, and sum pooling, each with its own strengths and weaknesses. Pooling is commonly used in convolutional neural networks (CNNs) to reduce the number of parameters and computations required for a CNN, making it more efficient and easier to train.

Unhelpful Answer: Pooling in deep learning is when you take the average of a bunch of pixels in a row or column. It helps make the network more efficient by reducing the number of parameters and computations required. There are different types of pooling like max pooling, average pooling, and sum pooling. It's used in CNNs to reduce 

In [37]:
query = "what is Maximum Likelihood Estimation?"
result = chain.invoke({"question": query, "chat_history": chat_history})
print(result["answer"])

 Maximum Likelihood Estimation (MLE) is a method of estimating the parameters of a statistical model given observed data. In the context of Bidirectional RNNs, MLE can be used to estimate the model's parameters by maximizing the likelihood of the observed sequence of input and output values given the model's parameters. The bidirectional RNN is trained using a variant of MLE called the forward-backward algorithm, which alternates between estimating the parameters of the forward RNN and the parameters of the backward RNN.
